In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import dataframe_image as dfi
import plotly.express as px

In [2]:
true_anomalies = sorted(pd.read_pickle('../Data_for_analysis/all_cases.pkl')['All_Cases'].tolist())
all_cases = sorted(list(set(pd.read_csv('../Data_for_analysis/small031.csv')['CaseID'].tolist())))
all_cases = [x+1 for x in all_cases]
anomalous_cases = sorted(list(set(pd.read_pickle('../Data_for_analysis/anomalous_cases_and_types.pkl')['Anomalous_Case'].tolist())))
anomaly_types = pd.read_pickle('../Data_for_analysis/anomalous_cases_and_types.pkl')['Anomaly_Type'].tolist()

binetv1 = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_binetv1.pkl')['Binetv1'].tolist())
binetv2 = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_binetv2.pkl')['Binetv2'].tolist())
binetv3 = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_binetv3.pkl')['Binetv3'].tolist())
dae = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_dae.pkl')['DAE'].tolist())
like_p = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_likelihood+.pkl')['Likelihood_Plus'].tolist())
naive = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_naive+.pkl')['Naive_Plus'].tolist())
random = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_random.pkl')['Random'].tolist())
sample = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_sampling.pkl')['Sampling'].tolist())
svm = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_svm.pkl')['SVM'].tolist())
tsp = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_tsp.pkl')['TSP'].tolist())
like = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_boehmer.pkl')['Boehmer'].tolist())
edbn = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_edbn.pkl')['EDBN'].tolist())
nerfcm = sorted(list(set(pd.read_csv('../Data_for_analysis/cases_detected_anomalous_nerfcm.csv')['0'].tolist())))
stat_lev = sorted(pd.read_pickle('../Data_for_analysis/cases_detected_anomalous_stat_lev.pkl')['Stat_Lev'].tolist())

In [3]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [5]:
df = pd.read_pickle('../Data_for_analysis/anomalous_cases_and_types.pkl')

cases_per_anomaly = dict()
for anomaly_type in sorted(list(set(df['Anomaly_Type'].to_list()))):
#     print(anomaly_type)
    cases_with_this_type_anomaly = []
    for i in range(len(df)):
        if df['Anomaly_Type'][i] == anomaly_type:
            if df['Anomalous_Case'][i] not in cases_with_this_type_anomaly:
                cases_with_this_type_anomaly.append(df['Anomalous_Case'][i])
    cases_per_anomaly[anomaly_type] = cases_with_this_type_anomaly

In [6]:
f1scores_per_method = dict()

anomaly_type_names = cases_per_anomaly.keys()
# all_algo_names = [binetv1, binetv2, binetv3, dae, like, like_p, naive, random, sample, svm, tsp, boehmer, edbn, nerfcm, stat_lev]
all_algo_names = [binetv1, binetv2, binetv3, dae, like, edbn, naive, sample, tsp, nerfcm, stat_lev]

for algo_name in all_algo_names:
    list_of_tuples = []
    for anomaly_type in anomaly_type_names:

        predicted_anomalous = algo_name
        actual_anomalous = cases_per_anomaly[anomaly_type]
        predicted_normal = list(set(cases_per_anomaly[anomaly_type])-set(predicted_anomalous))
        actual_normal = list(set(cases_per_anomaly[anomaly_type])-set(actual_anomalous))

        tp = len((set(predicted_anomalous).intersection(actual_anomalous)))
        tn = len((set(predicted_normal).intersection(actual_normal)))
        fp = len((set(predicted_normal).intersection(actual_anomalous)))
        fn = len((set(predicted_anomalous).intersection(actual_normal)))

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        accuracy = (tp+tn)/(tp+tn+fp+fn)
        f1score = (2*tp)/(2*tp+fp+fn)
        
        list_of_tuples.append((anomaly_type, f1score))
    f1scores_per_method[namestr(algo_name, globals())[0]] = list_of_tuples
    
f1scores_per_method

{'binetv1': [('Attribute', 1.0),
  ('Early', 1.0),
  ('Insert', 1.0),
  ('Late', 1.0),
  ('Rework', 1.0),
  ('Shift', 1.0),
  ('Skip', 1.0)],
 'binetv2': [('Attribute', 1.0),
  ('Early', 0.9693251533742331),
  ('Insert', 1.0),
  ('Late', 0.8823529411764706),
  ('Rework', 0.9844789356984479),
  ('Shift', 0.9264248704663213),
  ('Skip', 0.9241071428571429)],
 'binetv3': [('Attribute', 1.0),
  ('Early', 1.0),
  ('Insert', 1.0),
  ('Late', 1.0),
  ('Rework', 1.0),
  ('Shift', 1.0),
  ('Skip', 1.0)],
 'dae': [('Attribute', 0.67828418230563),
  ('Early', 0.8623024830699775),
  ('Insert', 0.6191780821917808),
  ('Late', 0.8728813559322034),
  ('Rework', 0.3284671532846715),
  ('Shift', 0.8677595628415301),
  ('Skip', 0.5123456790123457)],
 'like': [('Attribute', 0.3648424543946932),
  ('Early', 0.41009463722397477),
  ('Insert', 0.36893203883495146),
  ('Late', 0.3730886850152905),
  ('Rework', 0.36428571428571427),
  ('Shift', 0.391304347826087),
  ('Skip', 0.2536231884057971)],
 'edbn': [('

In [7]:
f1check = dict()
for key in f1scores_per_method.keys():
    temp_dict = dict()
    temp_list_algo_names = []
    temp_list_f1scores = []
    for item in f1scores_per_method[key]:
        temp_list_algo_names.append(item[0])
        temp_list_f1scores.append(item[1])
    temp_dict['theta'] = temp_list_algo_names
    temp_dict['r'] = temp_list_f1scores
    f1check[key] = temp_dict

In [8]:
# all_algo_names = [binetv1, binetv2, binetv3, dae, like, like_p, naive, random, sample, svm, tsp, boehmer, edbn, nerfcm, stat_lev, true_anomalies]
all_algo_names = [binetv1, binetv2, binetv3, dae, like, edbn, naive, sample, tsp, nerfcm, stat_lev, true_anomalies]
all_algo_names_str = []
for algo_name in all_algo_names:
    all_algo_names_str.append(namestr(algo_name, globals())[0])
complete_K = []

for i in range(len(all_algo_names)):
    
    breakout = 0
    temp_K = []
        
    for j in range(len(all_algo_names)):    
        print(all_algo_names_str[i], all_algo_names_str[j])

        algo1 = all_algo_names[i]     #list of cases detected anomalous by algo1
        algo2 = all_algo_names[j]     #list of cases detected anomalous by algo2

        #print(all_algo_names_str[i], all_algo_names_str[j])

        A = 0     #case in algo1 and in algo2
        B = 0     #case in algo1 and NOT in algo2
        C = 0     #case NOT in algo1 and in algo2
        D = 0     #case NOT in algo1 and NOT in algo2
        K = 0     #Kappa coefficient

        for case in all_cases:
            if case in algo1 and case in algo2:
                A += 1
            elif case in algo1 and case not in algo2:
                B += 1
            elif case not in algo1 and case in algo2:
                C += 1
            elif case not in algo1 and case not in algo2:
                D += 1

        print(A, B)
        print(C, D)

        Po = (A+D)/(A+B+C+D) #Probability of Agreement
        print('Po        ', Po)

        Pcorrect = ((A+B)/(A+B+C+D)) * ((A+C)/(A+B+C+D))
        print('Pcorrect  ', Pcorrect)

        Pincorrect = ((C+D)/(A+B+C+D)) * ((B+D)/(A+B+C+D))
        print('Pincorrect', Pincorrect)

        Pe = Pcorrect + Pincorrect #Probability of Random Agreement
        print('Pe        ', Pe)
        
        numerator = 1-Po
        denominator = 1-Pe
        
        K = 1-(numerator)/(denominator)
        print('K         ', K)
        temp_K.append(K)
    complete_K.append(temp_K)
    

binetv1 binetv1
1481 0
0 3519
Po         1.0
Pcorrect   0.08773444000000001
Pincorrect 0.49533444
Pe         0.58306888
K          1.0
binetv1 binetv2
1369 112
0 3519
Po         0.9776
Pcorrect   0.08109956
Pincorrect 0.51109956
Pe         0.59219912
K          0.9450712318227465
binetv1 binetv3
1481 0
0 3519
Po         1.0
Pcorrect   0.08773444000000001
Pincorrect 0.49533444
Pe         0.58306888
K          1.0
binetv1 dae
778 703
0 3519
Po         0.8594
Pcorrect   0.04608872
Pincorrect 0.59428872
Pe         0.64037744
K          0.6090345388787624
binetv1 like
322 1159
487 3032
Po         0.6708
Pcorrect   0.04792516
Pincorrect 0.58992516
Pe         0.63785032
K          0.09098359551222024
binetv1 edbn
318 1163
17 3502
Po         0.764
Pcorrect   0.019845400000000003
Pincorrect 0.6566454
Pe         0.6764908000000001
K          0.270499880683455
binetv1 naive
1481 0
0 3519
Po         1.0
Pcorrect   0.08773444000000001
Pincorrect 0.49533444
Pe         0.58306888
K          1.0
binet

121 688
403 3788
Po         0.7818
Pcorrect   0.016956640000000002
Pincorrect 0.7503566399999999
Pe         0.7673132799999999
K          0.062258473539014925
like true_anomalies
322 487
1159 3032
Po         0.6708
Pcorrect   0.04792516
Pincorrect 0.58992516
Pe         0.63785032
K          0.09098359551222024
edbn binetv1
318 17
1163 3502
Po         0.764
Pcorrect   0.019845400000000003
Pincorrect 0.6566454
Pe         0.6764908000000001
K          0.270499880683455
edbn binetv2
294 41
1075 3590
Po         0.7768
Pcorrect   0.0183446
Pincorrect 0.6775446
Pe         0.6958892000000001
K          0.26605697660194894
edbn binetv3
318 17
1163 3502
Po         0.764
Pcorrect   0.019845400000000003
Pincorrect 0.6566454
Pe         0.6764908000000001
K          0.270499880683455
edbn dae
180 155
598 4067
Po         0.8494
Pcorrect   0.010425199999999999
Pincorrect 0.7878252000000001
Pe         0.7982504000000001
K          0.2535301185231591
edbn like
297 38
512 4153
Po         0.89
Pcorrect   

42 105
1198 3655
Po         0.7394
Pcorrect   0.007291199999999999
Pincorrect 0.7298912
Pe         0.7371824
K          0.008437791076396417
nerfcm nerfcm
147 0
0 4853
Po         1.0
Pcorrect   0.00086436
Pincorrect 0.94206436
Pe         0.94292872
K          1.0
nerfcm stat_lev
13 134
511 4342
Po         0.871
Pcorrect   0.00308112
Pincorrect 0.8688811200000001
Pe         0.8719622400000001
K          -0.007515282991518379
nerfcm true_anomalies
49 98
1432 3421
Po         0.694
Pcorrect   0.00870828
Pincorrect 0.68310828
Pe         0.69181656
K          0.007084871270175763
stat_lev binetv1
524 0
957 3519
Po         0.8086
Pcorrect   0.03104176
Pincorrect 0.63004176
Pe         0.66108352
K          0.43525909392190065
stat_lev binetv2
512 12
857 3619
Po         0.8262
Pcorrect   0.02869424
Pincorrect 0.6500942399999999
Pe         0.6787884799999999
K          0.45892351557005207
stat_lev binetv3
524 0
957 3519
Po         0.8086
Pcorrect   0.03104176
Pincorrect 0.63004176
Pe         0.6

In [9]:
Kappa_df = pd.DataFrame.from_records(complete_K)
Kappa_df.columns = all_algo_names_str
Kappa_df.index = all_algo_names_str
Kappa_df

,binetv1,binetv2,binetv3,dae,like,edbn,naive,sample,tsp,nerfcm,stat_lev,true_anomalies
binetv1,1.000000,0.945071,1.000000,0.609035,0.090984,0.270500,1.000000,1.000000,0.878676,0.007085,0.435259,1.000000
binetv2,0.945071,1.000000,0.945071,0.541537,0.092791,0.266057,0.945071,0.945071,0.877201,0.009406,0.458924,0.945071
binetv3,1.000000,0.945071,1.000000,0.609035,0.090984,0.270500,1.000000,1.000000,0.878676,0.007085,0.435259,1.000000
dae,0.609035,0.541537,0.609035,1.000000,0.076570,0.253530,0.609035,0.609035,0.567432,0.004838,0.244906,0.609035
like,0.090984,0.092791,0.090984,0.076570,1.000000,0.468904,0.090984,0.090984,0.101191,0.000474,0.062258,0.090984
edbn,0.270500,0.266057,0.270500,0.253530,0.468904,1.000000,0.270500,0.270500,0.248318,0.022284,0.189892,0.270500
naive,1.000000,0.945071,1.000000,0.609035,0.090984,0.270500,1.000000,1.000000,0.878676,0.007085,0.435259,1.000000
sample,1.000000,0.945071,1.000000,0.609035,0.090984,0.270500,1.000000,1.000000,0.878676,0.007085,0.435259,1.000000
tsp,0.878676,0.877201,0.878676,0.567432,0.101191,0.248318,0.878676,0.878676,1.000000,0.008438,0.519977,0.878676
nerfcm,0.007085,0.009406,0.007085,0.004838,0.000474,0.022284,0.007085,0.007085,0.008438,1.000000,-0.007515,0.007085


In [10]:
# Kappa_df.to_excel('Kappa_df.xlsx')
Kappa_df.to_csv('../Analysis_Results/overall_kappa/Overall_Kappa_df.csv')